### *This file allows to generate all data necessary for all figures*

# **Useful packages and functions**

In [5]:
using DifferentialEquations, Plots, Polynomials, LaTeXStrings, ColorSchemes, DelimitedFiles
using Statistics, StatsPlots, Random, ProgressMeter, Printf, LinearAlgebra
include("FYON_2022_DA_kinetics.jl") # Loading of DA kinetics of gating variables
include("FYON_2022_DA_models.jl") # Loading of DA model
include("FYON_2022_DA_utils.jl") # Loading of some utils functions
include("FYON_2022_DA_gs_derivatives.jl") # Loading of X_inf derivatives
include("FYON_2022_DA_DIC.jl") # Loading of the DIC and compensation algorithm
include("FYON_2022_DA_neuromodulation.jl"); # Loading of the neuromodulation cells functions

# **Global variables**

In [6]:
# Definition of simulation time (in ms)
const Tfinal = 10000
const tspan  = (0.0, Tfinal)

# Definition of reversal potential values (in mV), [Mg] and membrane capacitance
const VNa   = 60. # Sodium reversal potential
const VK    = -85. # Potassium reversal potential
const VCa   = 60. # Calcium reversal potential
const VNMDA = 0. # NMDA reversal potential
const Vleak = -50. # Reversal potential of leak channels
const Mg    = 1.4 # Mg concentration
const C     = 1. # Membrane capacitance

# Definition of voltage range for the DICs
const Vmin = -60 
const Vmax = 0
const V    = range(Vmin, stop=Vmax, step=0.01)

# Definition of the number of cells in the random set
const ncells = 200;

# **Generating low frequency tonic spiking neurons using Monte Carlo (Figure 1)**

In [ ]:
# Initializing some variables
Na_max = 60
Kd_max = 20
CaL_max = 0.1
CaN_max = 0.3
ERG_max = 0.25
leak_max = 0.02
N = 3e5
VV_maxmax = 58.5
VV_maxmin = 54
VV_minmin = -80.5
VV_minmax = -77.5
f_max = 1.95
f_min = 1.75;

In [ ]:
# Fixing random seed
Random.seed!(226)

# Initializing a flag
flag_start = true

@showprogress 1 "Computing..." for i = 1 : N
    # Generating random conductances
    gleak = 0.005 + 0.015 * rand(1, 1)[1]
    gCaL = 0.95 * CaL_max * rand(1, 1)[1]#0.02 + (CaL_max-0.025) * rand(1, 1)[1]
    gKd = 0.95 * Kd_max * rand(1, 1)[1]#0.95 * (Kd_max-5) * rand(1, 1)[1]
    gNMDA = gleak * (0.12) / 0.013
    gNa = 0.95 * Na_max * rand(1, 1)[1]
    gCaN = 0.95 * CaN_max * rand(1, 1)[1]
    gERG = 0.95 * ERG_max * rand(1, 1)[1]
    
    # Parameter vector for simulations
    p = [0., gNa, gKd, gCaL, gCaN, gERG, gNMDA, gleak, C]

    # Initial conditions
    V0 = -90.
    x0 = [V0, m_inf(V0), h_inf(V0), n_inf(V0), mCaL_inf(V0), mCaN_inf(V0), 0., 0.]

    # Simulation
    prob = ODEProblem(DA_ODE, x0, tspan, p) # Describing the problem
    sol = solve(prob, saveat=0.1, verbose=false, maxiters=10) # Solving the problem

    # Saving results
    VV = sol[1, 20001:end]
    tt = sol.t[20001:end]

    # Computing spiking frequency
    if length(tt) < 1
        f = 0.
    elseif tt[end] != Tfinal
        f = 0.
    else
        f = extract_frequency(VV, tt)
        if isnan(f)
            f = 0.
        end
    end
    
    # Saving if spiking frequency is in the right order of magnitude
    if (f <= f_max && f >= f_min && maximum(VV) <= VV_maxmax && maximum(VV) >= VV_maxmin &&
        minimum(VV) <= VV_minmax && minimum(VV) >= VV_minmin)
        if flag_start
            # Creating the conductance matrix
            global g_all_MC = [gNa, gKd, gCaL, gCaN, gERG, gNMDA, gleak]
            flag_start = false
        else
            # Adding a line to the conductance matrix
            g_all_MC = hcat(g_all_MC, [gNa, gKd, gCaL, gCaN, gERG, gNMDA, gleak])

            # If we reach ncells neurons, break the loop
            if size(g_all_MC)[2] >= ncells
                break
            end
        end
    end
end

# Transposing the conductance matrix
if @isdefined g_all_MC
    g_all_MC = g_all_MC'
end;

In [ ]:
writedlm("./data/g_all_MC.dat", g_all_MC);

# **Generating low frequency tonic spiking neurons using DICs method (Figure 4A), separated and not**

In [7]:
# Definition of the number of cells in the random set
const ncells = 500;

In [9]:
# Fixing random seed
Random.seed!(226);

# Initial firing pattern
guth = 5.
Vth = -55.5
(g_all_init, ICs_th_init) = degeneracy_fixDICs_neuromod(ncells, 0.5, guth, Vth)
# create a spiking set with max variability in gCaS and gA

# Neuromodulating the initial set to the same value of gs_th (convention in the codes)
(g_all_spiking, ICs_th_spiking) = neuromodCaLCaN(ncells, g_all_init, ICs_th_init, 0.5, guth);

In [10]:
writedlm("./data/g_all_spiking.dat", g_all_spiking);

In [11]:
# Fixing random seed
Random.seed!(226)

# Initial firing pattern
(g_all_init_DIC, ICs_th_init_DIC) = degeneracy_fixDICs_neuromodDIC(ncells, 0.5, guth, Vth)
# create a spiking set with max variability in gCaS and gA

# Neuromodulating the initial set to obtain spiking, triplets and bursting
(g_all_spiking_DIC, ICs_th_spiking_DIC) = neuromodCaLCaN(ncells, g_all_init_DIC,
                                                           ICs_th_init_DIC, 0.5, guth);

In [58]:
writedlm("./data/g_all_spiking_DIC.dat", g_all_spiking_DIC);

In [13]:
# Fixing random seed
Random.seed!(226)

# Initial firing pattern
(g_all_init_leak, ICs_th_init_leak) = degeneracy_fixDICs_neuromodleak(ncells, 0.5, guth, Vth)
# create a spiking set with max variability in gCaS and gA

# Neuromodulating the initial set to obtain spiking, triplets and bursting
(g_all_spiking_leak, ICs_th_spiking_leak) = neuromodCaLCaN(ncells, g_all_init_leak,
                                                           ICs_th_init_leak, 0.5, guth);

In [59]:
writedlm("./data/g_all_spiking_leak.dat", g_all_spiking_leak);

# **Generating many low frequency tonic spiking neurons using DICs method (Figure 4B), not separated**

In [17]:
# Definition of the number of cells in the random set
const ncells = 2000;

In [18]:
# Fixing random seed
Random.seed!(226);

# Initial firing pattern
guth = 5.
Vth = -55.5
(g_all_init_many, ICs_th_init_many) = degeneracy_fixDICs_neuromod(ncells, 0.5, guth, Vth)
# create a spiking set with max variability in gCaS and gA

# Neuromodulating the initial set to the same value of gs_th (convention in the codes)
(g_all_spiking_many, ICs_th_spiking_many) = neuromodCaLCaN(ncells, g_all_init_many, 
                                                           ICs_th_init_many, 0.5, guth);

In [19]:
writedlm("./data/g_all_spiking_many.dat", g_all_spiking_many);

# **Generating neuromodulated neurons using DICs method (Figure 5), separated and not**

In [23]:
# Definition of the number of cells in the random set
const ncells = 500;

In [24]:
(g_all_triplets, ICs_th_triplets) = neuromodCaLCaN(ncells, g_all_init, ICs_th_init, -1.5, guth)

(g_all_bursting, ICs_th_bursting) = neuromodCaLCaN(ncells, g_all_init, ICs_th_init, -4., guth);

In [25]:
writedlm("./data/g_all_triplets.dat", g_all_triplets)

writedlm("./data/g_all_bursting.dat", g_all_bursting);

In [28]:
(g_all_triplets_DIC, ICs_th_triplets_DIC) = neuromodCaLCaN(ncells, g_all_init_DIC,
                                                           ICs_th_init_DIC, -1.5, guth)

(g_all_bursting_DIC, ICs_th_bursting_DIC) = neuromodCaLCaN(ncells, g_all_init_DIC, 
                                                           ICs_th_init_DIC, -4., guth);

In [29]:
writedlm("./data/g_all_triplets_DIC.dat", g_all_triplets_DIC)

writedlm("./data/g_all_bursting_DIC.dat", g_all_bursting_DIC);

In [30]:
(g_all_triplets_leak, ICs_th_triplets_leak) = neuromodCaLCaN(ncells, g_all_init_leak,
                                                           ICs_th_init_leak, -1.5, guth)

(g_all_bursting_leak, ICs_th_bursting_leak) = neuromodCaLCaN(ncells, g_all_init_leak, 
                                                           ICs_th_init_leak, -4., guth);

In [31]:
writedlm("./data/g_all_triplets_leak.dat", g_all_triplets_leak)

writedlm("./data/g_all_bursting_leak.dat", g_all_bursting_leak);

# **Computing burstiness of neuromodulated neurons (Figure 5)**

In [32]:
## Verifying robustness of the neuromodulated bursting sets
# Initializing variables
burstiness_triplets = zeros(ncells)
nb_spikes_triplets = zeros(ncells)
intraburst_f_triplets = zeros(ncells)
interburst_f_triplets = zeros(ncells)

# Loop over all neurons in the set
@showprogress 1 "Computing first part..." for i = 1 : ncells
  # Extracting the maximal ion channel conductances
  (gNa, gKd, gCaL, gCaN, gERG, gNMDA, gleak) = g_all_triplets[i, :]

  # Input current definition
  Iapp = 0.

  # Parameter vector for simulations
  p = [Iapp, gNa, gKd, gCaL, gCaN, gERG, gNMDA, gleak, C]

  # Initial conditions
  V0 = -90.
  x0 = [V0, m_inf(V0), h_inf(V0), n_inf(V0), mCaL_inf(V0), mCaN_inf(V0), 0., 0.]
    
  # Simulation
  prob = ODEProblem(DA_ODE, x0, tspan, p)
  sol = solve(prob, saveat=0.1, verbose=false)

  # Removing transient part
  V_t = sol[1, :][5001 : end]
  t = sol.t[5001 : end]
    
  # Computing the burstiness and burst characteristics of the firing pattern
  if t[end] != 10000.0
    (burstiness_triplets[i], nb_spikes_triplets[i], intraburst_f_triplets[i], 
        interburst_f_triplets[i]) = (NaN, NaN, NaN, NaN)
  else
    (burstiness_triplets[i], nb_spikes_triplets[i], intraburst_f_triplets[i], 
        interburst_f_triplets[i]) = extract_burstiness(V_t, t)
  end
end

# Removing unsuccessful simulations (only fews)
burstiness_triplets = burstiness_triplets[.!isnan.(burstiness_triplets)]
nb_spikes_triplets = nb_spikes_triplets[.!isnan.(nb_spikes_triplets)]
intraburst_f_triplets = intraburst_f_triplets[.!isnan.(intraburst_f_triplets)]
interburst_f_triplets = interburst_f_triplets[.!isnan.(interburst_f_triplets)]

# Initializing variables
burstiness_bursting = zeros(ncells)
nb_spikes_bursting = zeros(ncells)
intraburst_f_bursting = zeros(ncells)
interburst_f_bursting = zeros(ncells)

# Loop over all neurons in the set
@showprogress 1 "Computing second part..." for i = 1 : ncells
  # Extracting the maximal ion channel conductances
  (gNa, gKd, gCaL, gCaN, gERG, gNMDA, gleak) = g_all_bursting[i, :]

  # Input current definition
  Iapp = 0.

  # Parameter vector for simulations
  p = [Iapp, gNa, gKd, gCaL, gCaN, gERG, gNMDA, gleak, C]

  # Initial conditions
  V0 = -90.
  x0 = [V0, m_inf(V0), h_inf(V0), n_inf(V0), mCaL_inf(V0), mCaN_inf(V0), 0., 0.]
    
  # Simulation
  prob = ODEProblem(DA_ODE, x0, tspan, p)
  sol = solve(prob, saveat=0.1, verbose=false)

  # Removing transient part
  V_t = sol[1, :][5001 : end]
  t = sol.t[5001 : end]
    
  # Computing the burstiness and burst characteristics of the firing pattern
  if t[end] != 10000.0
    (burstiness_bursting[i], nb_spikes_bursting[i], intraburst_f_bursting[i], 
        interburst_f_bursting[i]) = (NaN, NaN, NaN, NaN)
  else
    (burstiness_bursting[i], nb_spikes_bursting[i], intraburst_f_bursting[i], 
        interburst_f_bursting[i]) = extract_burstiness(V_t, t)
  end
end

# Removing unsuccessful simulations (only fews)
burstiness_bursting = burstiness_bursting[.!isnan.(burstiness_bursting)]
nb_spikes_bursting = nb_spikes_bursting[.!isnan.(nb_spikes_bursting)]
intraburst_f_bursting = intraburst_f_bursting[.!isnan.(intraburst_f_bursting)]
interburst_f_bursting = interburst_f_bursting[.!isnan.(interburst_f_bursting)];

Computing first part...100%|████████████████████████████| Time: 0:01:48
Computing second part...100%|███████████████████████████| Time: 0:01:27


In [33]:
writedlm("./data/burstiness_triplets.dat", burstiness_triplets)

writedlm("./data/burstiness_bursting.dat", burstiness_bursting);

In [34]:
## Verifying robustness of the neuromodulated bursting sets
# Initializing variables
burstiness_triplets_DIC = zeros(ncells)
nb_spikes_triplets_DIC = zeros(ncells)
intraburst_f_triplets_DIC = zeros(ncells)
interburst_f_triplets_DIC = zeros(ncells)

# Loop over all neurons in the set
@showprogress 1 "Computing first part..." for i = 1 : ncells
  # Extracting the maximal ion channel conductances
  (gNa, gKd, gCaL, gCaN, gERG, gNMDA, gleak) = g_all_triplets_DIC[i, :]

  # Input current definition
  Iapp = 0.

  # Parameter vector for simulations
  p = [Iapp, gNa, gKd, gCaL, gCaN, gERG, gNMDA, gleak, C]

  # Initial conditions
  V0 = -90.
  x0 = [V0, m_inf(V0), h_inf(V0), n_inf(V0), mCaL_inf(V0), mCaN_inf(V0), 0., 0.]
    
  # Simulation
  prob = ODEProblem(DA_ODE, x0, tspan, p)
  sol = solve(prob, saveat=0.1, verbose=false)

  # Removing transient part
  V_t = sol[1, :][5001 : end]
  t = sol.t[5001 : end]
    
  # Computing the burstiness and burst characteristics of the firing pattern
  if t[end] != 10000.0
    (burstiness_triplets_DIC[i], nb_spikes_triplets_DIC[i], intraburst_f_triplets_DIC[i], 
        interburst_f_triplets_DIC[i]) = (NaN, NaN, NaN, NaN)
  else
    (burstiness_triplets_DIC[i], nb_spikes_triplets_DIC[i], intraburst_f_triplets_DIC[i], 
        interburst_f_triplets_DIC[i]) = extract_burstiness(V_t, t)
  end
end

# Removing unsuccessful simulations (only fews)
burstiness_triplets_DIC = burstiness_triplets_DIC[.!isnan.(burstiness_triplets_DIC)]
nb_spikes_triplets_DIC = nb_spikes_triplets_DIC[.!isnan.(nb_spikes_triplets_DIC)]
intraburst_f_triplets_DIC = intraburst_f_triplets_DIC[.!isnan.(intraburst_f_triplets_DIC)]
interburst_f_triplets_DIC = interburst_f_triplets_DIC[.!isnan.(interburst_f_triplets_DIC)]

# Initializing variables
burstiness_bursting_DIC = zeros(ncells)
nb_spikes_bursting_DIC = zeros(ncells)
intraburst_f_bursting_DIC = zeros(ncells)
interburst_f_bursting_DIC = zeros(ncells)

# Loop over all neurons in the set
@showprogress 1 "Computing second part..." for i = 1 : ncells
  # Extracting the maximal ion channel conductances
  (gNa, gKd, gCaL, gCaN, gERG, gNMDA, gleak) = g_all_bursting_DIC[i, :]

  # Input current definition
  Iapp = 0.

  # Parameter vector for simulations
  p = [Iapp, gNa, gKd, gCaL, gCaN, gERG, gNMDA, gleak, C]

  # Initial conditions
  V0 = -90.
  x0 = [V0, m_inf(V0), h_inf(V0), n_inf(V0), mCaL_inf(V0), mCaN_inf(V0), 0., 0.]
    
  # Simulation
  prob = ODEProblem(DA_ODE, x0, tspan, p)
  sol = solve(prob, saveat=0.1, verbose=false)

  # Removing transient part
  V_t = sol[1, :][5001 : end]
  t = sol.t[5001 : end]
    
  # Computing the burstiness and burst characteristics of the firing pattern
  if t[end] != 10000.0
    (burstiness_bursting_DIC[i], nb_spikes_bursting_DIC[i], intraburst_f_bursting_DIC[i], 
        interburst_f_bursting_DIC[i]) = (NaN, NaN, NaN, NaN)
  else
    (burstiness_bursting_DIC[i], nb_spikes_bursting_DIC[i], intraburst_f_bursting_DIC[i], 
        interburst_f_bursting_DIC[i]) = extract_burstiness(V_t, t)
  end
end

# Removing unsuccessful simulations (only fews)
burstiness_bursting_DIC = burstiness_bursting_DIC[.!isnan.(burstiness_bursting_DIC)]
nb_spikes_bursting_DIC = nb_spikes_bursting_DIC[.!isnan.(nb_spikes_bursting_DIC)]
intraburst_f_bursting_DIC = intraburst_f_bursting_DIC[.!isnan.(intraburst_f_bursting_DIC)]
interburst_f_bursting_DIC = interburst_f_bursting_DIC[.!isnan.(interburst_f_bursting_DIC)];

Computing first part...100%|████████████████████████████| Time: 0:00:58
Computing second part...100%|███████████████████████████| Time: 0:01:28


In [35]:
writedlm("./data/burstiness_triplets_DIC.dat", burstiness_triplets_DIC)

writedlm("./data/burstiness_bursting_DIC.dat", burstiness_bursting_DIC);

In [36]:
## Verifying robustness of the neuromodulated bursting sets
# Initializing variables
burstiness_triplets_leak = zeros(ncells)
nb_spikes_triplets_leak = zeros(ncells)
intraburst_f_triplets_leak = zeros(ncells)
interburst_f_triplets_leak = zeros(ncells)

# Loop over all neurons in the set
@showprogress 1 "Computing first part..." for i = 1 : ncells
  # Extracting the maximal ion channel conductances
  (gNa, gKd, gCaL, gCaN, gERG, gNMDA, gleak) = g_all_triplets_leak[i, :]

  # Input current definition
  Iapp = 0.

  # Parameter vector for simulations
  p = [Iapp, gNa, gKd, gCaL, gCaN, gERG, gNMDA, gleak, C]

  # Initial conditions
  V0 = -90.
  x0 = [V0, m_inf(V0), h_inf(V0), n_inf(V0), mCaL_inf(V0), mCaN_inf(V0), 0., 0.]
    
  # Simulation
  prob = ODEProblem(DA_ODE, x0, tspan, p)
  sol = solve(prob, saveat=0.1, verbose=false)

  # Removing transient part
  V_t = sol[1, :][5001 : end]
  t = sol.t[5001 : end]
    
  # Computing the burstiness and burst characteristics of the firing pattern
  if t[end] != 10000.0
    (burstiness_triplets_leak[i], nb_spikes_triplets_leak[i], intraburst_f_triplets_leak[i], 
        interburst_f_triplets_leak[i]) = (NaN, NaN, NaN, NaN)
  else
    (burstiness_triplets_leak[i], nb_spikes_triplets_leak[i], intraburst_f_triplets_leak[i], 
        interburst_f_triplets_leak[i]) = extract_burstiness(V_t, t)
  end
end

# Removing unsuccessful simulations (only fews)
burstiness_triplets_leak = burstiness_triplets_leak[.!isnan.(burstiness_triplets_leak)]
nb_spikes_triplets_leak = nb_spikes_triplets_leak[.!isnan.(nb_spikes_triplets_leak)]
intraburst_f_triplets_leak = intraburst_f_triplets_leak[.!isnan.(intraburst_f_triplets_leak)]
interburst_f_triplets_leak = interburst_f_triplets_leak[.!isnan.(interburst_f_triplets_leak)]

# Initializing variables
burstiness_bursting_leak = zeros(ncells)
nb_spikes_bursting_leak = zeros(ncells)
intraburst_f_bursting_leak = zeros(ncells)
interburst_f_bursting_leak = zeros(ncells)

# Loop over all neurons in the set
@showprogress 1 "Computing second part..." for i = 1 : ncells
  # Extracting the maximal ion channel conductances
  (gNa, gKd, gCaL, gCaN, gERG, gNMDA, gleak) = g_all_bursting_leak[i, :]

  # Input current definition
  Iapp = 0.

  # Parameter vector for simulations
  p = [Iapp, gNa, gKd, gCaL, gCaN, gERG, gNMDA, gleak, C]

  # Initial conditions
  V0 = -90.
  x0 = [V0, m_inf(V0), h_inf(V0), n_inf(V0), mCaL_inf(V0), mCaN_inf(V0), 0., 0.]
    
  # Simulation
  prob = ODEProblem(DA_ODE, x0, tspan, p)
  sol = solve(prob, saveat=0.1, verbose=false)

  # Removing transient part
  V_t = sol[1, :][5001 : end]
  t = sol.t[5001 : end]
    
  # Computing the burstiness and burst characteristics of the firing pattern
  if t[end] != 10000.0
    (burstiness_bursting_leak[i], nb_spikes_bursting_leak[i], intraburst_f_bursting_leak[i], 
        interburst_f_bursting_leak[i]) = (NaN, NaN, NaN, NaN)
  else
    (burstiness_bursting_leak[i], nb_spikes_bursting_leak[i], intraburst_f_bursting_leak[i], 
        interburst_f_bursting_leak[i]) = extract_burstiness(V_t, t)
  end
end

# Removing unsuccessful simulations (only fews)
burstiness_bursting_leak = burstiness_bursting_leak[.!isnan.(burstiness_bursting_leak)]
nb_spikes_bursting_leak = nb_spikes_bursting_leak[.!isnan.(nb_spikes_bursting_leak)]
intraburst_f_bursting_leak = intraburst_f_bursting_leak[.!isnan.(intraburst_f_bursting_leak)]
interburst_f_bursting_leak = interburst_f_bursting_leak[.!isnan.(interburst_f_bursting_leak)];

Computing first part...100%|████████████████████████████| Time: 0:00:50
Computing second part...100%|███████████████████████████| Time: 0:01:03


In [37]:
writedlm("./data/burstiness_triplets_leak.dat", burstiness_triplets_leak)

writedlm("./data/burstiness_bursting_leak.dat", burstiness_bursting_leak);

# **Computing neurons along neuromodulation paths (Figure 6)**

In [39]:
# Fixing random seed
Random.seed!(224)

# Initializing some variables
gsths = 0.5 : -0.005 : -4.
n = 40
gCaLs = zeros(length(gsths), n)
gCaNs = zeros(length(gsths), n)
burstiness = zeros(length(gsths), n)

# Input current definition
Iapp = 0.

# Initial firing pattern
(g_all_init, ICs_th_init) = degeneracy_fixDICs_neuromod(n, gsths[1], guth, Vth)

# Looping over all neuromodulated states
@showprogress 1 "Computing..." for i = 1 : length(gsths)
    # Neuromodulating the initial set
    (g_all, ICs_th) = neuromodCaLCaN(n, g_all_init, ICs_th_init, gsths[i], guth)
    
    gCaLs[i, :] = g_all[:, 3]
    gCaNs[i, :] = g_all[:, 4]
    
    # Looping over the set to compute the burstiness
    for j = 1 : n
        # Extracting the maximal ion channel conductances
        (gNa, gKd, gCaL, gCaN, gERG, gNMDA, gleak) = g_all[j, :]

        # Parameter vector for simulations
        p = [Iapp, gNa, gKd, gCaL, gCaN, gERG, gNMDA, gleak, C]

        # Initial conditions
        V0 = -90.
        x0 = [V0, m_inf(V0), h_inf(V0), n_inf(V0), mCaL_inf(V0), mCaN_inf(V0), 0., 0.]

        # Simulation
        prob = ODEProblem(DA_ODE, x0, tspan, p)
        sol = solve(prob, saveat=0.1, verbose=false)

        # Removing transient part
        V_t = sol[1, :][5001 : end]
        t = sol.t[5001 : end]

        # Computing the burstiness and burst characteristics of the firing pattern
        if t[end] != 10000.0
            (burstiness[i, j], _, _, _) = (0, NaN, NaN, NaN)
        else
            (burstiness[i, j], _, _, _) = extract_burstiness(V_t, t)
        end

        # If spiking
        if isnan(burstiness[i, j])
            burstiness[i, j] = 0.
        end
    end
end

Computing...100%|███████████████████████████████████████| Time: 1:01:31


In [42]:
writedlm("./data/gCaLs_paths.dat", gCaLs)

writedlm("./data/gCaNs_paths.dat", gCaNs)

writedlm("./data/burstiness_paths.dat", burstiness)

writedlm("./data/g_all_init_paths.dat", g_all_init);

# **Computing low frequency tonic spiking neurons using homogeneous scaling (Figure supp 1)**

In [50]:
# Definition of the number of cells in the random set
const ncells = 200

g_all_MC = readdlm("./data/g_all_MC.dat");

In [52]:
# Number of ion channels in the DA model
nb_channels = 6

# Initializing some variables
g_all_MC_wout_NMDA = zeros(ncells, nb_channels)

# Removing NMDA channels from the PCA analysis since it is constant
j = 1
for i in [1, 2, 3, 4, 5, 7]
    g_all_MC_wout_NMDA[:, j] = deepcopy(g_all_MC[:, i])
    j = j + 1
end

# Means on spiking point cloud
mean_MC = zeros(1, nb_channels)
mean!(mean_MC, g_all_MC_wout_NMDA)

# Defining scaling factors and limits for the plots
s1 = 2
s2 = 2
Na_max = 60
Kd_max = 20
CaL_max = 0.1
CaN_max = 0.3
ERG_max = 0.25
leak_max = 0.02

# Defining scaling factors for homogeneous scaling
gNa_scaling = mean_MC[1] / mean_MC[6]
gKd_scaling = mean_MC[2] / mean_MC[6]
gCaL_scaling = mean_MC[3] / mean_MC[6]
gCaN_scaling = mean_MC[4] / mean_MC[6]
gERG_scaling = mean_MC[5] / mean_MC[6];

In [53]:
# Fixing random seed
Random.seed!(225)

# Initializing a flag
flag_start = true

for i = 1 : 200
    # Generating random conductances
    gleak = 0.006 + 0.0125 * rand(1, 1)[1]
    gCaL = gleak * gCaL_scaling
    gKd = gleak * gKd_scaling
    gNMDA = gleak * (0.12) / 0.013
    gNa = gleak * gNa_scaling
    gCaN = gleak * gCaN_scaling
    gERG = gleak * gERG_scaling
        
    if flag_start
        # Creating the conductance matrix
        global g_all_MC_leak = [gNa, gKd, gCaL, gCaN, gERG, gNMDA, gleak]
        flag_start = false
    else
        # Adding a line to the conductance matrix
        g_all_MC_leak = hcat(g_all_MC_leak, [gNa, gKd, gCaL, gCaN, gERG, gNMDA, gleak])
    end
end

# Transposing the conductance matrix
if @isdefined g_all_MC_leak
    g_all_MC_leak = g_all_MC_leak'
end;

In [54]:
# Input current definition
Iapp = 0. : -0.01 : -1.

# Initializing some variables
f_leak = zeros(length(Iapp), ncells)

@showprogress 1 "Computing..." for (k, Iapp_i) in enumerate(Iapp)
    # Looping over all neurons
    for i = 1 : ncells
        # Retrieving maximal conductances from what had been computed
        (gNa, gKd, gCaL, gCaN, gERG, gNMDA, gleak) = g_all_MC_leak[i, :]

        # Parameter vector for simulations
        p = [Iapp_i, gNa, gKd, gCaL, gCaN, gERG, gNMDA, gleak, C]

        # Initial conditions
        V0 = -90.
        x0 = [V0, m_inf(V0), h_inf(V0), n_inf(V0), mCaL_inf(V0), mCaN_inf(V0), 0., 0.]

        # Simulation
        prob = ODEProblem(DA_ODE, x0, tspan, p) # Describing the problem
        sol = solve(prob, saveat=0.1) # Solving the problem

        # Saving results
        VV = sol[1, 50001:end]
        tt = sol.t[50001:end]

        # Computing spiking frequency
        if tt[end] != Tfinal
            f_leak[k, i] = 0.
        else
            f_leak[k, i] = extract_frequency(VV, tt)
            if isnan(f_leak[k, i])
                f_leak[k, i] = 0.
            end
        end
    end
end

Computing...100%|███████████████████████████████████████| Time: 0:12:58


In [56]:
# Initializing some variables
rheobase = zeros(ncells)

# Computing the rheobase for each neuon
for i = 1 : ncells
    f_leak_i = f_leak[:, i]
    rheobase[i] = maximum(Iapp[findall(f_leak[:, i] .<= 0.)])
end

In [60]:
writedlm("./data/g_all_MC_leak.dat", g_all_MC_leak)

writedlm("./data/f_leak.dat", f_leak)

writedlm("./data/Iapp.dat", Iapp)

writedlm("./data/rheobase.dat", rheobase);